In [35]:
context = []
response = []

file = open('/mnt/d/downloads/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt',errors='ignore')
for i,line in enumerate(file):
    l = line.split("+++$+++")
    if i%2==0:
        context.append(l[-1])
    else:
        response.append("awwal "+l[-1]+" aakhir")

context = context[:-1]

print(len(response),len(context))

152356 152356


In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
MAX_SEQ_LENGTH = 50
EMBEDDING_DIM = 50
vocab_size = 20000
tokenizer = Tokenizer(num_words=vocab_size,oov_token="<oov>")
tokenizer.fit_on_texts(context+response)

word_index = {word:index for word,index in tokenizer.word_index.items() if index<vocab_size }
index_to_word = { index:word for word,index in word_index.items()}

context_seq = tokenizer.texts_to_sequences(context)
response_seq = tokenizer.texts_to_sequences(response)

In [38]:
encoder_inputs = []
decoder_inputs = []
decoder_outputs = []
for i in range(len(context_seq)):
    if not(len(context_seq[i])>MAX_SEQ_LENGTH or len(response_seq[i])-1>MAX_SEQ_LENGTH):
        encoder_inputs.append(context_seq[i])
        decoder_inputs.append(response_seq[i][:-1])
        decoder_outputs.append(response_seq[i][1:])

encoder_inputs = pad_sequences(encoder_inputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')
decoder_inputs = pad_sequences(decoder_inputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')
decoder_outputs = pad_sequences(decoder_outputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')

print(len(encoder_inputs),len(decoder_inputs),len(decoder_outputs))

147875 147875 147875


In [39]:
print(decoder_outputs[:5])

[[ 47  20   7   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 65 113   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [965   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 30   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 39  25 946  11 987   1   3   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   

In [40]:
import os
import numpy as np
embeddings_index = {}
f = open(os.path.join('/mnt/d/glove.6B', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(1,EMBEDDING_DIM)

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            mask_zero = True,
                            trainable=False)

Found 400000 word vectors.


In [41]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model

In [42]:
def encoder_decoder_model():
    hidden_dim = 64
    #encoder
    encoder_inputs = Input(shape=(None,))
    emb_en_input = embedding_layer(encoder_inputs)
    encoder_lstm = LSTM(hidden_dim,return_state=True)
    encoder_outputs,state_h,state_c = encoder_lstm(emb_en_input)
    encoder_states = [state_h,state_c]

    #decoder
    decoder_inputs = Input(shape=(None,))
    emb_de_input = embedding_layer(decoder_inputs)
    decoder_lstm = LSTM(hidden_dim,return_state=True,return_sequences=True)
    decoder_outputs,_,_ = decoder_lstm(emb_de_input,initial_state=encoder_states)
    decoder_dense  = Dense(vocab_size,activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    #model
    model = Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
    return model

In [43]:
model = encoder_decoder_model()
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])

In [44]:
print(encoder_inputs.shape)

(147875, 50)


In [48]:
def data_generator(batch_size = 64):
    index = 0
    while True:
        if(len(encoder_inputs)>=(index+1)*batch_size):
            index = 0
        X1  = encoder_inputs[index*batch_size:(index+1)*batch_size]
        X2  = decoder_inputs[index*batch_size:(index+1)*batch_size]
        Y  = decoder_outputs[index*batch_size:(index+1)*batch_size]
        index+=1
        yield([X1,X2],Y)

In [49]:
model.fit(data_generator(),steps_per_epoch=len(decoder_inputs)//64,epochs=1)

   5/2310 [..............................] - ETA: 5:43:25 - loss: 1.7216 - accuracy: 0.0073

KeyboardInterrupt: 